# Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_path = '../adanagyhazifeladat2019/public_train_trx.csv'
test_path = '../adanagyhazifeladat2019/public_test_trx.csv'

In [3]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
alldf = pd.concat([train_df, test_df], sort=True)
print("Alldf: ", alldf.shape)
target = 'TARGET_successful_purchase'
cust_df = alldf.groupby('session_id', as_index=False).agg({target:'min',
                                                          'test_or_train_flag':'min'})
print("Cust_df: ", cust_df.shape)
sorted_df = alldf.sort_values(['session_id', 'duration_of_session', 'click_num'])
sorted_df = sorted_df.reset_index(drop=True)

Alldf:  (418970, 22)
Cust_df:  (49086, 3)


# Merge previous ideas together:
* Nagyházi kezdetek 1-2
* kisházi példamegoldás

### 1. Data from last row of the session:

In [4]:
sessions_df_last = sorted_df.drop_duplicates('session_id', keep='last').copy()
# 0 clicknumokat eldobjuk, így biztosan lesz kattintás az első sessionben
sorted_df=sorted_df[sorted_df['click_num']!=0].copy()
sessions_df_first= sorted_df.drop_duplicates('session_id', keep='first').copy()
print("sessions_df_last: ",sessions_df_last.shape)
print("sessions_df_first: ", sessions_df_first.shape)

sessions_df_last:  (49086, 22)
sessions_df_first:  (48930, 22)


In [5]:
sessions_df_last['datetime']=sessions_df_last['start_date_of_session']+" "+sessions_df_last['start_time_of_session']
sessions_df_last['datetime']=sessions_df_last['datetime'].apply(lambda x: pd.to_datetime(x))
sessions_df_last['weekday']=sessions_df_last['datetime'].dt.dayofweek
#sessions_df_last['hour']=sessions_df_last['datetime'].apply(lambda x: x.hour)
sessions_df_last['hour']=sessions_df_last['datetime'].dt.hour
sessions_df_last['minute']=sessions_df_last['datetime'].dt.minute

In [6]:
def calc_click(row):
    if row['click_num'] == 0:
        return row['duration_of_session']
    else:
        return row['duration_of_session']/row['click_num']

In [7]:
sessions_df_last['click_avg']= sessions_df_last.apply(calc_click, axis=1)

## Extra: 

In [8]:
def calc_sum_price_ratio(row):
    if row['sum_price_of_visited_products'] == 0:
        return(2)
    else:
        return row['sum_price_of_products_in_basket']/row['sum_price_of_visited_products']

In [9]:
sessions_df_last['sum_price_ratio'] = sessions_df_last.apply(calc_sum_price_ratio, axis=1)

In [10]:
def calc_max_price_ratio(row):
    if(row['maximum_price_of_visited_products']==0):
        return(-1)
    else:
        return row['price_of_more_expensive_product_in_basket']/row['maximum_price_of_visited_products']

In [11]:
sessions_df_last['max_price_ratio'] = sessions_df_last.apply(calc_max_price_ratio, axis=1)

In [12]:
def calc_min_price_ratio(row):
    if(row['minimum_price_of_visited_products']==0):
        return(-1)
    else:
        return row['price_of_cheapest_product_in_basket']/row['minimum_price_of_visited_products']

In [13]:
sessions_df_last['min_price_ratio'] = sessions_df_last.apply(calc_min_price_ratio, axis=1)

In [14]:
def calc_mean_price_in_basket(row):
    if(row['basket_element_number']==0):
        return(0)
    else:
        return row['sum_price_of_products_in_basket']/row['basket_element_number']

In [15]:
sessions_df_last['mean_price_in_basket'] = sessions_df_last.apply(calc_mean_price_in_basket, axis=1)

In [16]:
sessions_df_last['new_cust_value'] = sessions_df_last['customer_value']*sessions_df_last['level_of_purchasing_process']

---

In [17]:
last_colnames = list(sessions_df_last.columns)
last_colnames.remove('test_or_train_flag')
last_colnames.remove('TARGET_successful_purchase')
last_colnames.remove('start_date_of_session')
last_colnames.remove('start_time_of_session')
last_colnames.remove('datetime')
features_last = sessions_df_last[last_colnames].copy()
print("features_last: ", features_last.shape)

features_last:  (49086, 27)


In [18]:
for i in range(len(last_colnames)):
    if(last_colnames[i]!="session_id"):
        last_colnames[i]="last_"+last_colnames[i]
last_colnames

['last_basket_element_number',
 'last_click_num',
 'last_customer_age',
 'last_customer_value',
 'last_duration_of_session',
 'last_last_order_of_customer',
 'last_level_of_purchasing_process',
 'last_lifetime_customer_account',
 'last_max_val',
 'last_maximum_price_of_visited_products',
 'last_minimum_price_of_visited_products',
 'last_num_of_previous_payments',
 'last_price_of_cheapest_product_in_basket',
 'last_price_of_more_expensive_product_in_basket',
 'last_regio_of_customer',
 'session_id',
 'last_sum_price_of_products_in_basket',
 'last_sum_price_of_visited_products',
 'last_weekday',
 'last_hour',
 'last_minute',
 'last_click_avg',
 'last_sum_price_ratio',
 'last_max_price_ratio',
 'last_min_price_ratio',
 'last_mean_price_in_basket',
 'last_new_cust_value']

In [19]:
features_last.columns = last_colnames

In [20]:
features_last.isna().sum()

last_basket_element_number                            0
last_click_num                                        0
last_customer_age                                 21372
last_customer_value                               21659
last_duration_of_session                              0
last_last_order_of_customer                       21342
last_level_of_purchasing_process                  13638
last_lifetime_customer_account                    21659
last_max_val                                      21659
last_maximum_price_of_visited_products             1058
last_minimum_price_of_visited_products             1058
last_num_of_previous_payments                     21342
last_price_of_cheapest_product_in_basket           1378
last_price_of_more_expensive_product_in_basket     1378
last_regio_of_customer                            21342
session_id                                            0
last_sum_price_of_products_in_basket               1378
last_sum_price_of_visited_products              

In [21]:
features_last.head().T

,4,10,31,35,47
last_basket_element_number,1,3,12,1,4
last_click_num,1,4,60,13,56
last_customer_age,NaN,42,31,35,NaN
last_customer_value,NaN,491,548,504,NaN
last_duration_of_session,49.166,384.522,1981.1,1471.28,1271.37
last_last_order_of_customer,NaN,64,42,15,NaN
last_level_of_purchasing_process,1,1,1,1,1
last_lifetime_customer_account,NaN,204,45,138,NaN
last_max_val,NaN,600,300,1300,NaN
last_maximum_price_of_visited_products,49.99,39.99,89.99,54.99,49.95


In [22]:
cust_df = cust_df.merge(features_last, on='session_id', how='left')
cust_df.head().T

,0,1,2,3,4
session_id,exxxxxshop.si-0.00021101047198248773,exxxxxshop.si-0.0002712991769484001,exxxxxshop.si-0.00033158788092821383,exxxxxshop.si-0.00039187658370279567,exxxxxshop.si-0.0004521652850530127
TARGET_successful_purchase,NaN,NaN,NaN,NaN,NaN
test_or_train_flag,1,1,1,1,1
last_basket_element_number,1,3,12,1,4
last_click_num,1,4,60,13,56
last_customer_age,NaN,42,31,35,NaN
last_customer_value,NaN,491,548,504,NaN
last_duration_of_session,49.166,384.522,1981.1,1471.28,1271.37
last_last_order_of_customer,NaN,64,42,15,NaN
last_level_of_purchasing_process,1,1,1,1,1


In [23]:
cust_df.isna().sum()

session_id                                            0
TARGET_successful_purchase                        24502
test_or_train_flag                                    0
last_basket_element_number                            0
last_click_num                                        0
last_customer_age                                 21372
last_customer_value                               21659
last_duration_of_session                              0
last_last_order_of_customer                       21342
last_level_of_purchasing_process                  13638
last_lifetime_customer_account                    21659
last_max_val                                      21659
last_maximum_price_of_visited_products             1058
last_minimum_price_of_visited_products             1058
last_num_of_previous_payments                     21342
last_price_of_cheapest_product_in_basket           1378
last_price_of_more_expensive_product_in_basket     1378
last_regio_of_customer                          

In [24]:
df_na = cust_df[cust_df['TARGET_successful_purchase'].isna()]
df_na.shape

(24502, 29)

In [25]:
df_na.isna().sum()

session_id                                            0
TARGET_successful_purchase                        24502
test_or_train_flag                                    0
last_basket_element_number                            0
last_click_num                                        0
last_customer_age                                 10694
last_customer_value                               10835
last_duration_of_session                              0
last_last_order_of_customer                       10677
last_level_of_purchasing_process                   6762
last_lifetime_customer_account                    10835
last_max_val                                      10835
last_maximum_price_of_visited_products              564
last_minimum_price_of_visited_products              564
last_num_of_previous_payments                     10677
last_price_of_cheapest_product_in_basket            719
last_price_of_more_expensive_product_in_basket      719
last_regio_of_customer                          

##  2. Aggregálós módszer - órai 2.

In [26]:
sorted_df = alldf.sort_values(['session_id', 'duration_of_session', 'click_num'])

In [27]:
# szám változók
aggregalando_valtozok = list(alldf.columns)
aggregalando_valtozok.remove('test_or_train_flag')
aggregalando_valtozok.remove(target)
aggregalando_valtozok.remove('start_date_of_session')
aggregalando_valtozok.remove('start_time_of_session')
aggregalando_valtozok.remove('session_id')
aggregalando_valtozok

['basket_element_number',
 'click_num',
 'customer_age',
 'customer_value',
 'duration_of_session',
 'last_order_of_customer',
 'level_of_purchasing_process',
 'lifetime_customer_account',
 'max_val',
 'maximum_price_of_visited_products',
 'minimum_price_of_visited_products',
 'num_of_previous_payments',
 'price_of_cheapest_product_in_basket',
 'price_of_more_expensive_product_in_basket',
 'regio_of_customer',
 'sum_price_of_products_in_basket',
 'sum_price_of_visited_products']

In [28]:
for aggregalos_modszer in ['min', 'max', 'mean']:
    task = {}
    ujoszlopnevek = []
    for v in aggregalando_valtozok:
        task[v] = aggregalos_modszer
        ujoszlopnevek.append(aggregalos_modszer+"_"+v) # mindegyik változónévre kivéve session_id
    stat = alldf.groupby(['session_id'], as_index=False).agg(task)
    stat.columns = ['session_id']+ujoszlopnevek
    cust_df = cust_df.merge(stat, on='session_id', how='left')

Extra:

In [29]:
stat = alldf.groupby(['session_id'], as_index=False).agg({'click_num' : 'count'})

In [30]:
stat.columns=['session_id', 'num_sessions']
stat.head()

,session_id,num_sessions
0,exxxxxshop.si-0.00021101047198248773,5
1,exxxxxshop.si-0.0002712991769484001,6
2,exxxxxshop.si-0.00033158788092821383,21
3,exxxxxshop.si-0.00039187658370279567,4
4,exxxxxshop.si-0.0004521652850530127,12


In [31]:
cust_df = cust_df.merge(stat, on='session_id', how='left')

In [32]:
cust_df.shape

(49086, 81)

In [33]:
cust_df.isna().sum()

session_id                                            0
TARGET_successful_purchase                        24502
test_or_train_flag                                    0
last_basket_element_number                            0
last_click_num                                        0
                                                  ...  
mean_price_of_more_expensive_product_in_basket     1378
mean_regio_of_customer                            21342
mean_sum_price_of_products_in_basket               1378
mean_sum_price_of_visited_products                 1058
num_sessions                                          0
Length: 81, dtype: int64

## 3. kishazi ötletből

In [34]:
sessions_df_first['click_avg_first']=sessions_df_first.apply(calc_click, axis=1)

In [35]:
sessions_df_first['click_avg_first'].mean()

53.014220336941314

In [36]:
cust_df = cust_df.merge(sessions_df_first[['session_id', 'click_avg_first']], on='session_id', how='left')

!!! fillna(-1)

In [37]:
cust_df=cust_df.fillna(-1)

In [38]:
cust_df.shape

(49086, 82)

In [39]:
cust_df.columns

Index(['session_id', 'TARGET_successful_purchase', 'test_or_train_flag',
       'last_basket_element_number', 'last_click_num', 'last_customer_age',
       'last_customer_value', 'last_duration_of_session',
       'last_last_order_of_customer', 'last_level_of_purchasing_process',
       'last_lifetime_customer_account', 'last_max_val',
       'last_maximum_price_of_visited_products',
       'last_minimum_price_of_visited_products',
       'last_num_of_previous_payments',
       'last_price_of_cheapest_product_in_basket',
       'last_price_of_more_expensive_product_in_basket',
       'last_regio_of_customer', 'last_sum_price_of_products_in_basket',
       'last_sum_price_of_visited_products', 'last_weekday', 'last_hour',
       'last_minute', 'last_click_avg', 'last_sum_price_ratio',
       'last_max_price_ratio', 'last_min_price_ratio',
       'last_mean_price_in_basket', 'last_new_cust_value',
       'min_basket_element_number', 'min_click_num', 'min_customer_age',
       'min_custom

In [40]:
cust_df.to_csv("cust_df_tuesday2.csv", index=False)